In [48]:
import pandas as pd

cargando el dataset de accientes de aviones y revisando si posee algun valor nulo del total de filas

In [49]:
df = pd.read_csv('./AccidentesAviones.csv')
print(df.shape)
print(df.isnull().sum())

(5008, 18)
Unnamed: 0                0
fecha                     0
HORA declarada            0
Ruta                      0
OperadOR                  0
flight_no                 0
route                     0
ac_type                   0
registration              0
cn_ln                     0
all_aboard                0
PASAJEROS A BORDO         0
crew_aboard               0
cantidad de fallecidos    0
passenger_fatalities      0
crew_fatalities           0
ground                    0
summary                   0
dtype: int64


evaluando la cantidad de valores faltantes que estarian compuestos por el caracter de interrogacion

In [50]:
faltantes = []

import plotly.express as px

faltantes = []

for i in df.columns:
    try:
        faltante =  df['Unnamed: 0'][df[i]=='?'].count()
    except:
        faltante = 0
    faltantes.append(faltante)

df_faltantes = pd.DataFrame()
df_faltantes['Columnas'] = df.columns
df_faltantes['Faltantes'] = faltantes
df_faltantes
fig = px.bar(df_faltantes, x='Columnas', y='Faltantes')
fig.show()

evaluando el tipo de datos que posee cada columna para su posterior tratamiento

In [51]:
df.dtypes

Unnamed: 0                 int64
fecha                     object
HORA declarada            object
Ruta                      object
OperadOR                  object
flight_no                 object
route                     object
ac_type                   object
registration              object
cn_ln                     object
all_aboard                object
PASAJEROS A BORDO         object
crew_aboard               object
cantidad de fallecidos    object
passenger_fatalities      object
crew_fatalities           object
ground                    object
summary                   object
dtype: object

cambiando el nombre a las columnas existentes hasta el momento

In [52]:
df.columns = ['index', 'fecha', 'hora_declarada', 'lugar_accidente', 'operador', 'vuelo_nro', 'ruta', 'tipo_avion', 'registro', 'cn_ln', 'total_a_bordo', 'pasajeros_a_bordo', 'tripulacion_a_bordo', 'cantidad_de_fallecidos',
           'pasajeros_fallecidos', 'tripulacion_fallecidos', 'fallecidos_en_tierra', 'resumen']

In [53]:
df['tipo_avion']

0                    Wright Flyer III
1                      Wright Byplane
2                           Dirigible
3                    Curtiss seaplane
4              Zeppelin L-1 (airship)
                    ...              
5003      Eurocopter AS350B3 Ecureuil
5004    Beechcraft B300 King Air 350i
5005                 Beechcraft 1900D
5006         Lockheed C-130H Hercules
5007               Antonov An 26B-100
Name: tipo_avion, Length: 5008, dtype: object

cambiando el formato literal de fecha a datetime

In [54]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['fecha'].head()

0   1908-09-17
1   1909-09-07
2   1912-07-12
3   1913-08-06
4   1913-09-09
Name: fecha, dtype: datetime64[ns]

para normalizar la columna de hora usamos una funcion personalizada y agregamos replaces para quitar caracteres ajenos a la hora

In [55]:
def normalize_hour(value):
    value = value.replace('?', '00:00')
    value = value.replace('c ', '')
    value = value.replace('Z', '')
    value = value.replace('c:', '')
    value = value.replace(' ', '')
    value = value.replace(':', '')
    value = value.replace(';', '')
    
    if ':' not in value:
        return value
    elif value[0] == ':':
        return f'00{value}'
    elif value[-1] == ':':
        return f'{value}00'
    elif len(value) == 4:
        return '0' + value
    else:
        return value
    
df['hora_declarada'] = df['hora_declarada'].apply(lambda x: normalize_hour(x))
df['hora_declarada'] = pd.to_datetime(df['hora_declarada'], format='%H%M').dt.strftime('%H:%M')

df['hora_declarada'].head()

0    17:18
1    00:00
2    06:30
3    00:00
4    18:30
Name: hora_declarada, dtype: object

agregando la columna de anio del accidente

In [56]:
df['anio'] = df['fecha'].dt.year

agregando la columna de mes del accidente

In [57]:
df['mes'] = df['fecha'].dt.month

agregando la columna de dia del accidente

In [58]:
df['dia'] = df['fecha'].dt.day

para normalizar la columna paises haremos uso de un dataset adicional, que contiene informacion de los paises, ciudades y capitales del mundo

In [59]:
df2 = pd.read_csv('./cities.csv')

list1 = df2['country_name'].str.lower().tolist()
list2 = df2['state_name'].str.lower().tolist()
list3 = df2['name'].str.lower().tolist()

In [60]:
print(len(list1))
print(len(list2))
print(len(list3))

150890
150890
150890


armamos una funcion personalizada que nos permita buscar el nombre de la ciudad y nos permita retornar el nombre del pais, ya que el dataset de accidentes posee informacion desde los 1900 en el transcurso de ese anio hasta la fecha hay paises que ya dejaron de serlo, por lo mismo se encontrara merma en esta columna

opte por usar try except y no ciclos for por que la carga computacional es demasiada, ya que la opcion a usar este dataset seria conectarse a una API y hacer consultas para obtener los datos de las columnas de ubicacion faltantes pero estos tiene un limite de consultas y el tiempo de espera para que termine es muy tardado

In [61]:
def get_country(value):
    new_value = value.split(', ')
    try:
        index = list1.index(new_value[-1].lower())
    except:
        try:
            index = list2.index(new_value[-1].lower())
        except:
            try:
                index = list3.index(new_value[-1].lower())
            except:
                return 'no encontrado'
    return list1[index].title()
    
get_country('Off West Hartlepool, England')

'United Kingdom'

quitamos las columnas que no tengan informacion ya que no tenemos opcion de obtenerla de otra columna

In [62]:
df = df[df['lugar_accidente']!='?']

In [63]:
df['lugar_accidente'] = df['lugar_accidente'].str.replace('. United Kingdom', ', United Kingdom')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('Bulgeria', 'Bulgaria')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('Unied Kingdom', 'United Kingdom')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('South of Gibraltar', 'Gibraltar')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('NY', 'New York')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('WY', 'Wyoming')
df['lugar_accidente'] = df['lugar_accidente'].str.replace(',Brazil', ', Brazil')
df['lugar_accidente'] = df['lugar_accidente'].str.replace('  ', ' ')

C:\Users\KhorneFlakes\AppData\Local\Temp\ipykernel_9628\1079818205.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



hacemos uso de la funcion que creamos lineas atras para obtener el nombre del pais

In [64]:
df['pais'] = df['lugar_accidente'].apply(lambda x: get_country(x))
df['pais']

0       United States
1              France
2       United States
3              Canada
4       no encontrado
            ...      
5003    United States
5004          Nigeria
5005          Myanmar
5006      Philippines
5007           Russia
Name: pais, Length: 5003, dtype: object

hacemos uso de la libreria pycountry para obtener el codigo en formato ISO 3166-1 alpha-3, para poder plotear un mapa con la cantidad de accidentes en el mundo

In [65]:
import pycountry_convert as pc

country_code = pc.country_name_to_country_alpha2("China", cn_name_format="default")
print(country_code)

def get_code(value):
    try:
        code = pc.country_name_to_country_alpha3(value, cn_name_format="default")
        return code
    except:
        return 'no code'

df['code'] = df['pais'].apply(lambda x: get_code(x))
df['code']

CN


0           USA
1           FRA
2           USA
3           CAN
4       no code
         ...   
5003        USA
5004        NGA
5005        MMR
5006        PHL
5007        RUS
Name: code, Length: 5003, dtype: object

haciendo uso de la ultima libreria que usamos creamos una funcion que nos permita obtener el continente del pais en una nueva columna para usarla en una grafica en el dashboard futuro

In [66]:
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return 'no continent'

df['continent'] = df['pais'].apply(lambda x: country_to_continent(x))
df['continent']

0       North America
1              Europe
2       North America
3       North America
4        no continent
            ...      
5003    North America
5004           Africa
5005             Asia
5006             Asia
5007           Europe
Name: continent, Length: 5003, dtype: object

In [67]:
df.code.value_counts()

USA        1018
no code     602
RUS         256
BRA         175
COL         152
           ... 
LCA           1
LVA           1
SVN           1
BTN           1
FSM           1
Name: code, Length: 159, dtype: int64

In [68]:
df['pais'].value_counts()

United States    1018
no encontrado     600
Russia            256
Brazil            175
Colombia          152
                 ... 
Saint Lucia         1
Latvia              1
Bhutan              1
Oman                1
Micronesia          1
Name: pais, Length: 160, dtype: int64

hay varios caracteres que si bien no estan vacios equivalen a lo mismo, estos contienen el caracter de interrogacion

abajo vemos cuantos de estan por columna hasta esta etapa de la limpieza

In [69]:
for i in df.columns:
    print(i + ': ' + str(df[i][df[i]=='?'].count()))

index: 0
fecha: 0
hora_declarada: 0
lugar_accidente: 0
operador: 10
vuelo_nro: 3677
ruta: 758
tipo_avion: 13
registro: 269
cn_ln: 663
total_a_bordo: 17
pasajeros_a_bordo: 221
tripulacion_a_bordo: 219
cantidad_de_fallecidos: 8
pasajeros_fallecidos: 235
tripulacion_fallecidos: 235
fallecidos_en_tierra: 43
resumen: 59
anio: 0
mes: 0
dia: 0
pais: 0
code: 0
continent: 0


reemplazamos los valores faltantes con ceros

In [70]:
df = df.replace('?', 0)

cambiamos a formato numero las columnas con datos faltantes, esto para poder hacer operaciones y poder completarlas

In [71]:
df['total_a_bordo'] = pd.to_numeric(df['total_a_bordo'])
df['pasajeros_a_bordo'] = pd.to_numeric(df['pasajeros_a_bordo'])
df['tripulacion_a_bordo'] = pd.to_numeric(df['tripulacion_a_bordo'])
df['cantidad_de_fallecidos'] = pd.to_numeric(df['cantidad_de_fallecidos'])
df['pasajeros_fallecidos'] = pd.to_numeric(df['pasajeros_fallecidos'])
df['tripulacion_fallecidos'] = pd.to_numeric(df['tripulacion_fallecidos'])
df['fallecidos_en_tierra'] = pd.to_numeric(df['fallecidos_en_tierra'])

mediante una serie de ecuaciones hacemos uso de columnas numericas de las cuales si tenemos datos para completar otras columnas numericas

In [72]:
df['total_a_bordo'] = df.apply(lambda x: 
    (x['pasajeros_a_bordo'] + x['tripulacion_a_bordo']) if (x['total_a_bordo']==0) else x['total_a_bordo'], axis=1)

In [73]:
df['tripulacion_a_bordo'] = df.apply(lambda x: 
    (x['total_a_bordo'] - x['pasajeros_a_bordo']) if (x['tripulacion_a_bordo']==0) else x['tripulacion_a_bordo'], axis=1)

In [74]:
df['pasajeros_a_bordo'] = df.apply(lambda x: 
    (x['total_a_bordo'] - x['tripulacion_a_bordo']) if (x['pasajeros_a_bordo']==0) else x['pasajeros_a_bordo'], axis=1)

In [75]:
df['cantidad_de_fallecidos'] = df.apply(lambda x: 
    (x['pasajeros_fallecidos'] + x['tripulacion_fallecidos']) if (x['cantidad_de_fallecidos']==0) else (x['cantidad_de_fallecidos']), axis=1)

In [76]:
df['cantidad_de_fallecidos'] = df.apply(lambda x: 
    (x['pasajeros_fallecidos'] + x['tripulacion_fallecidos']) if (x['cantidad_de_fallecidos']==0) else (x['cantidad_de_fallecidos']), axis=1)

In [77]:
df['cantidad_de_fallecidos'] = df.apply(lambda x: 
    (x['pasajeros_fallecidos'] + x['tripulacion_fallecidos']) if (x['cantidad_de_fallecidos']==0) else (x['cantidad_de_fallecidos']), axis=1)

In [78]:
df['total_a_bordo'][df['total_a_bordo']==0].value_counts()
df['pasajeros_a_bordo'][df['pasajeros_a_bordo']==0].value_counts()

0    1085
Name: pasajeros_a_bordo, dtype: int64

revisamos que las columnas tratadas ya no posean valores faltantes

In [85]:
for i in df.columns:
    print(i + ': ' + str(df[i][df[i]=='?'].count()))

index: 0
fecha: 0
hora_declarada: 0
lugar_accidente: 0
operador: 0
vuelo_nro: 0
ruta: 0
tipo_avion: 0
registro: 0
cn_ln: 0
total_a_bordo: 0
pasajeros_a_bordo: 0
tripulacion_a_bordo: 0
cantidad_de_fallecidos: 0
pasajeros_fallecidos: 0
tripulacion_fallecidos: 0
fallecidos_en_tierra: 0
resumen: 0
anio: 0
mes: 0
dia: 0
pais: 0
code: 0
continent: 0


In [80]:
df.dtypes

index                              int64
fecha                     datetime64[ns]
hora_declarada                    object
lugar_accidente                   object
operador                          object
vuelo_nro                         object
ruta                              object
tipo_avion                        object
registro                          object
cn_ln                             object
total_a_bordo                      int64
pasajeros_a_bordo                  int64
tripulacion_a_bordo                int64
cantidad_de_fallecidos             int64
pasajeros_fallecidos               int64
tripulacion_fallecidos             int64
fallecidos_en_tierra               int64
resumen                           object
anio                               int64
mes                                int64
dia                                int64
pais                              object
code                              object
continent                         object
dtype: object

con la tabla con las columnas que vamos a utilizar ya tratadas hacemos uso de SQLAlchemy para exportar este dataset procesado a una base de datos

In [81]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///data_viz.db', echo=True)

sqlite_connection = engine.connect()

df.to_sql('accidentes', sqlite_connection, index=False, if_exists='replace')

sqlite_connection.close()